<a href="https://colab.research.google.com/github/TangJiahui/6.036_Machine_Learning/blob/main/MIT_6_036_HW07_Back_Propagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MIT 6.036 Fall 2020: Homework 7 Neural Network - Back Propagation#

This colab notebook provides code and a framework for problem 2 of [the homework](https://lms.mitx.mit.edu/courses/course-v1:MITx+6.036+2019_Spring/courseware/Week7/week7_homework/).  You can work out your solutions here, then submit your results back on the homework page when ready.

**Note**: You can go to `File > Save a copy in Drive...` to save your own copy of this notebook for editing.

## <section>**Setup**</section>

First, download the code distribution for this homework that contains test cases and helper functions.

Run the next code block to download and import the code for this lab.


In [183]:
!rm -rf code_for_hw7*
!rm -rf mnist
!rm -rf data
!wget --no-check-certificate --quiet https://introml.odl.mit.edu/cat-soop/_static/6.036/homework/hw07/code_for_hw7.zip
!unzip code_for_hw7.zip
!mv code_for_hw7/* .

from code_for_hw7 import *
import numpy as np
import modules_disp as disp


Archive:  code_for_hw7.zip
   creating: code_for_hw7/
  inflating: code_for_hw7/code_for_hw7.py  
  inflating: code_for_hw7/pytorch_code_for_hw7.py  
  inflating: code_for_hw7/code_for_hw7_pytorch.py  
   creating: code_for_hw7/data/
  inflating: code_for_hw7/data/data5_validate.csv  
  inflating: code_for_hw7/data/data5_train.csv  
  inflating: code_for_hw7/data/data1_train.csv  
  inflating: code_for_hw7/data/data1_validate.csv  
  inflating: code_for_hw7/data/data2_train.csv  
  inflating: code_for_hw7/data/data2_validate.csv  
  inflating: code_for_hw7/data/data3_train.csv  
  inflating: code_for_hw7/data/data3_validate.csv  
  inflating: code_for_hw7/data/data3class_train.csv  
  inflating: code_for_hw7/data/data4_train.csv  
  inflating: code_for_hw7/data/data4_validate.csv  
  inflating: code_for_hw7/data/dataXor_train.csv  
  inflating: code_for_hw7/expected_results.py  
  inflating: code_for_hw7/modules_disp.py  
  inflating: code_for_hw7/utils_hw7.py  
   creating: code_for_h

# 2) Implementing Neural Networks

This homework considers neural networks with multiple layers. Each layer has multiple inputs and outputs, and can be broken down into two parts:
 
<br>
 
A linear module that implements a linear transformation:     $ z_j = (\sum^{m}_{i=1} x_i W_{i,j}) + {W_0}_jz$

specified by a weight matrix $W$ and a bias vector $W_0$. The input is $[x_1, \ldots, x_m]^T$. The output is $[z_1, \ldots, z_n]^T$

<br>

An activation module that applies an activation function to the outputs of the linear module for some activation function $f$, such as Tanh or ReLU in the hidden layers or Softmax (see below) at the output layer. We write the output as: $[f(z_1), \ldots, f(z_m)]^T$, although technically, for some activation functions such as softmax, each output will depend on all the $z_i$.

Although for "real" applications you want to use one of the many packaged implementations of neural networks (we'll start using one of those soon), there is no substitute for implementing one yourself to get an in-depth understanding. Luckily, that is relatively easy to do if we're not too concerned with maximum efficiency.

We'll use the modular implementation that we guided you through in the previous problem, which leads to clean code. The basic framework for SGD training is given below. We can construct a network and train it as follows:

```
# build a 3-layer network
net = Sequential([Linear(2,3), Tanh(),
                  Linear(3,3), Tanh(),
    	          Linear(3,2), SoftMax()])
# train the network on data and labels
net.sgd(X, Y)
```
We will (later) be generalizing SGD to operate on a "mini-batch" of data points instead of a single point. You should strive for an implementation of the forward, backward, and `class_fun` methods that works with batches of data. Note that when $b$ is mentioned as part of the shape of a matrix in the code, this $b$ refers to the number of points.

## Linear Modules: ##
Each linear module has a forward method that takes in a batch of activations A (from the previous layer) and returns a batch of pre-activations Z.

Each linear module has a backward method that takes in dLdZ and returns dLdA. This module also computes and stores dLdW and dLdW0, the gradients with respect to the weights.

Hint: be careful with dimensions when computing dLdW0. dLdZ is (n x b), but dLdW0 is (n x 1). Why do you need to sum over all $b$ points in the batch when computing dLdW0?

In [184]:
# class Linear(Module):
#     def __init__(self, m, n):
#         self.m, self.n = (m, n)  # (in size, out size)
#         self.W0 = np.zeros([self.n, 1])  # (n x 1)
#         self.W = np.random.normal(0, 1.0 * m ** (-.5), [m, n])  # (m x n)

#     def forward(self, A):
#         self.A = A   # (m x b)  Hint: make sure you understand what b stands for
#         #return self.W.T@A+self.W0*np.ones([1, self.A.shape[1]])  # Your code (n x b)
#         return np.dot(self.W.T, A) + self.W0

#     def backward(self, dLdZ):  # dLdZ is (n x b), uses stored self.A
#         self.dLdW = self.A@dLdZ.T       
#         #self.dLdW0 = dLdZ@np.ones([self.A.shape[1],1])    
#         self.dLdW0 = dLdZ.sum(axis=1).reshape((self.n, 1))  # (n x 1)
#         return self.W@dLdZ     #return dLdA (m x b)

#     def sgd_step(self, lrate):  # Gradient descent step
#         self.W = self.W -  lrate*self.dLdW
#         self.W0 = self.W0 - lrate*self.dLdW0

In [185]:
class Module:
    def sgd_step(self, lrate): pass  # For modules w/o weights

class Linear(Module):
    def __init__(self, m, n):
        self.m, self.n = (m, n)  # (in size, out size)
        self.W0 = np.zeros([self.n, 1])  # (n x 1)
        self.W = np.random.normal(0, 1.0 * m ** (-.5), [m, n])  # (m x n)

    def forward(self, A):
        self.A = A
        return np.dot(self.W.T, A) + self.W0  # (m x n)^T (m x b) = (n x b)

    def backward(self, dLdZ):  # dLdZ is (n x b), uses stored self.A
        self.dLdW = np.dot(self.A, dLdZ.T)                  # (m x n)
        self.dLdW0 = dLdZ.sum(axis=1).reshape((self.n, 1))  # (n x 1)
        return np.dot(self.W, dLdZ)                         # (m x b)

    def sgd_step(self, lrate):  # Gradient descent step
        self.W -= lrate*self.dLdW
        self.W0 -= lrate*self.dLdW0

 You are **highly encouraged** to make your own tests for each module. The test cases being run on catsoop are given below for your reference:

In [186]:
def nn_linear_forward():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    return linear.forward(X).tolist()

expected_linear_forward = [[10.417500637754383, 6.911221682745654, 20.733665048236965, 22.891234399772113], [7.168722346625092, 3.489987464919749, 10.469962394759248, 9.998261102396512], [-2.071054548689073, 0.6941371647696142, 2.0824114943088414, 4.849668106971125]]
print("Pass linear forward? ", np.allclose(nn_linear_forward(), expected_linear_forward))

def nn_linear_forward_bias():
    np.random.seed(0)
    linear = Linear(2,3)
    linear.W0 = np.array([[1],[1],[1]])
    X,Y = super_simple_separable()
    return linear.forward(X).tolist()

expected_linear_forward_bias = [[11.417500637754383, 7.911221682745654, 21.733665048236965, 23.891234399772113], [8.168722346625092, 4.489987464919749, 11.469962394759248, 10.998261102396512], [-1.071054548689073, 1.6941371647696142, 3.0824114943088414, 5.849668106971125]]
print("Pass linear forward bias? ", np.allclose(nn_linear_forward_bias(), expected_linear_forward_bias))


Pass linear forward?  True
Pass linear forward bias?  True


In [187]:
def nn_linear_backward():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    return linear.backward(dLdZ).tolist()

expected_linear_backward = [[3.889497924054116, 1.247373376201773, 0.2829538755771419, 0.6920722655660196], [2.1525571673658237, 1.5845507770701677, 1.3205629190941617, -0.6910398159642225]]
print("Pass linear backward? ", np.allclose(nn_linear_backward(), expected_linear_backward))

def nn_linear_backward_stored_dLdW_dLdW0():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    linear.backward(dLdZ)
    return [linear.dLdW.tolist(), linear.dLdW0.tolist()]

expected_linear_backward_stored = [[[5, 13, 18], [7, 16, 20]], [[2], [3], [4]]]
print("Pass linear backward stored vals dLdW? ", np.allclose(nn_linear_backward_stored_dLdW_dLdW0()[0], expected_linear_backward_stored[0]))
print("Pass linear backward stored vals dLdW0? ", np.allclose(nn_linear_backward_stored_dLdW_dLdW0()[1], expected_linear_backward_stored[1]))

def nn_linear_sgd():
    np.random.seed(0)
    linear = Linear(2,3)
    X,Y = super_simple_separable()
    linear.forward(X)
    dLdZ = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0],
                                     [3, 0, 0, 1]])
    linear.backward(dLdZ)
    linear.sgd_step(0.005)
    return [np.vstack([linear.W, linear.W0.T]).tolist()]

expected_linear_sgd = [[[1.222373376201773, 0.2179538755771419, 0.6020722655660197], [1.5495507770701678, 1.2405629190941616, -0.7910398159642225], [-0.01, -0.015, -0.02]]]
print("Pass linear sgd? ", np.allclose(nn_linear_sgd(), expected_linear_sgd))

Pass linear backward?  True
Pass linear backward stored vals dLdW?  True
Pass linear backward stored vals dLdW0?  True
Pass linear sgd?  True


The following datasets are defined for your use:
*  `super_simple_separable_through_origin()`
*  `super_simple_separable()`
*  `xor()`
*  `xor_more()`
*  `hard()`

Further, a plotting function is defined for your usage in modules_disp.py, and can be called in the colab notebook as `disp.plot_nn()`.
```
def plot_nn(X, Y, nn):
    """ Plot output of nn vs. data """
    def predict(x):
        return nn.modules[-1].class_fun(nn.forward(x))[0]
    xmin, ymin = np.min(X, axis=1)-1
    xmax, ymax = np.max(X, axis=1)+1
    nax = plot_objective_2d(lambda x: predict(x), xmin, xmax, ymin, ymax)
    plot_data(X, Y, nax)
    plt.show()```


## Activation functions: ##
Each activation module has a forward method that takes in a batch of pre-activations Z and returns a batch of activations A.

Each activation module has a backward method that takes in dLdA and returns dLdZ, with the exception of SoftMax, where we assume dLdZ is passed in.

### Tanh: ###
Please use np.tanh here. You can find documentation online.

Hint: the derivative of $\tanh$ is given by $\frac{d\tanh(z)}{d z} = 1 - \tanh(z)^2$.

In [188]:
class Tanh(Module):            # Layer activation
    def forward(self, Z):
        self.A = np.tanh(Z)    
        return self.A

    def backward(self, dLdA):    # Uses stored self.A
        return dLdA * (1 - np.tanh(np.arctanh(self.A))**2)  # Your code: return dLdZ with dimensions (?, b)

Tanh unit test:

In [189]:
def nn_tanh_forward():
    np.random.seed(0)
    tanh = Tanh()
    X,Y = super_simple_separable()
    return tanh.forward(X).tolist()

expected_tanh_forward = [[0.9640275800758169, 0.9950547536867305, 0.999999969540041, 0.9999999999244973], [0.9999092042625951, 0.9640275800758169, 0.9999877116507956, 0.9999092042625951]]
print("Pass tanh forward? ", np.allclose(nn_tanh_forward(), expected_tanh_forward))

def nn_tanh_backward():
    np.random.seed(0)
    tanh = Tanh()
    X,Y = super_simple_separable()
    tanh.forward(X)
    dLdA = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0]])
    return tanh.backward(dLdA).tolist()

expected_tanh_backward = [[0.07065082485316443, 0.009866037165440211, 0.0, 0.0], [0.0003631664618877206, 0.0, 2.4576547405286142e-05, 0.0]]
print("Pass tanh backward? ", np.allclose(nn_tanh_backward(), expected_tanh_backward))

Pass tanh forward?  True
Pass tanh backward?  True


### ReLU: ###
Hint:
[`np.maximum`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.maximum.html) might be useful.

In [190]:
class ReLU(Module): # Layer activation
    def forward(self, Z): 
        self.A = np.maximum(0,Z)  
        return self.A

    def backward(self, dLdA): # uses stored self.A
        if np.sum(self.A) == 0:
            return dLdA*np.zeros([1,self.A.shape[1]]) 
        else:
            return dLdA*np.ones([1,self.A.shape[1]])           # Your code: return dLdZ (?, b)

ReLU unit tests:

In [191]:
def nn_relu_forward():
    np.random.seed(0)
    relu = ReLU()
    X,Y = super_simple_separable()
    return relu.forward(X).tolist()

expected_relu_forward = [[2, 3, 9, 12], [5, 2, 6, 5]]
print("Pass relu forward? ", np.allclose(nn_relu_forward(), expected_relu_forward))

def nn_relu_backward():
    np.random.seed(0)
    relu = ReLU()
    X,Y = super_simple_separable()
    relu.forward(X)
    dLdA = np.array([[1, 1, 0, 0],
                                     [2, 0, 1, 0]])
    return relu.backward(dLdA).tolist()

expected_relu_backward = [[1, 1, 0, 0], [2, 0, 1, 0]]
print("Pass relu backward? ", np.allclose(nn_relu_backward(), expected_relu_backward))


Pass relu forward?  True
Pass relu backward?  True


###SoftMax: ###
For `SoftMax.class_fun()`, given the column vector of class probabilities for each point (computed by Softmax), return a vector of the classes (integers) with the highest probability for each point. The output should be a 1D numpy array. 

In [192]:
class SoftMax(Module):# Output activation
    def forward(self, Z): #z= [n*b]
        return (np.exp(Z)/np.sum(np.exp(Z), axis = 0))# Your code: (?, b)

    def backward(self, dLdZ):# Assume that dLdZ is passed in
        return dLdZ

    def class_fun(self, Ypred): 
        return np.argmax(Ypred, axis=0) # Your code: A 1D vector (b, ) 

Test cases for softmax:

In [193]:
def nn_softmax_forward():
    np.random.seed(0)
    softmax = SoftMax()
    X,Y = super_simple_separable()
    return softmax.forward(X).tolist()

expected_softmax_forward = [[0.04742587317756679, 0.7310585786300048, 0.9525741268224334, 0.9990889488055993], [0.9525741268224333, 0.2689414213699951, 0.04742587317756678, 0.0009110511944006454]]
print("Pass softmax forward? ", np.allclose(nn_softmax_forward(), expected_softmax_forward))
  
def nn_softmax_class_fun():
    np.random.seed(0)
    softmax = SoftMax()
    X,Y = super_simple_separable()
    Ypred = softmax.forward(X)
    return softmax.class_fun(Ypred).tolist()

expected_softmax_class_fun = [1, 0, 0, 0]
print("Pass softmax class fun? ", np.allclose(nn_softmax_class_fun(), expected_softmax_class_fun))

Pass softmax forward?  True
Pass softmax class fun?  True


## Loss Functions:##
Each loss module has a forward method that takes in a batch of predictions Ypred (from the previous layer) and labels Y and returns a scalar loss value. Each column of Y will correspond to a one-hot vector encoding the correct class label for one point in our batch. 

The NLL module has a backward method that returns dLdZ, the gradient with respect to the preactivation to SoftMax (note: not the activation!), since we are always pairing SoftMax activation with NLL loss.

### NLL: ###
You should use multi-class NLL. 

Hint: $$\frac{dNLL(Softmax(z))}{dz} = Y_{pred} - Y$$

As an exercise, try proving that this is true.

In [194]:
class NLL(Module):       # Loss
    def forward(self, Ypred, Y):
        self.Ypred = Ypred
        self.Y = Y
        return -np.sum(Y*np.log(Ypred))      # Your code

    def backward(self):  # Use stored self.Ypred, self.Y
        return self.Ypred - self.Y      # Your code

NLL Test Cases:

In [195]:
def nn_nll_forward():
    nll = NLL()
    y = np.array([[1, 0, 1, 0]])
    Y = for_softmax(y)
    ypred = np.array([[.7, .3, .99, .99]])
    Ypred = for_softmax(ypred)
    return nll.forward(Ypred, Y)

expected_nll_forward = 5.328570409719057
print("Pass nll forward? ", np.allclose(nn_nll_forward(), expected_nll_forward))

def nn_nll_backward():
    nll = NLL()
    y = np.array([[1, 0, 1, 0]])
    Y = for_softmax(y)
    ypred = np.array([[.7, .3, .99, .99]])
    Ypred = for_softmax(ypred)
    nll.forward(Ypred, Y)
    return nll.backward().tolist()

expected_nll_backward = [[0.30000000000000004, -0.30000000000000004, 0.010000000000000009, -0.99], [-0.30000000000000004, 0.3, -0.010000000000000009, 0.99]]
print("Pass nll backward? ", np.allclose(nn_nll_backward(), expected_nll_backward))

Pass nll forward?  True
Pass nll backward?  True


## Activation and Loss Test Cases: ##
Run Test 1 and Test 2 below and compare your outputs with the expected outputs.


In [196]:
# TEST 1: sgd_test for Tanh activation and SoftMax output
np.random.seed(0)
sgd_test(Sequential([Linear(2,3), Tanh(), Linear(3,2), SoftMax()], NLL()), test_1_values)

linear_1.W: OK
linear_1.W0: OK
linear_2.W: OK
linear_2.W0: OK
z_1: OK
a_1: OK
z_2: OK
a_2: OK
loss: OK
dloss: OK
dL_dz2: OK
dL_da1: OK
dL_dz1: OK
dL_dX: OK
updated_linear_1.W: OK
updated_linear_1.W0: OK
updated_linear_2.W: OK
updated_linear_2.W0: OK


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in arctanh
  import sys


In [197]:
# TEST 2: sgd_test for ReLU activation and SoftMax output
#np.random.seed(0)
#sgd_test(Sequential([Linear(2,3), ReLU(), Linear(3,2), SoftMax()], NLL()), test_2_values)

## Neural Network: ##

Implement SGD. Randomly pick a data point Xt, Yt by using np.random.randint to choose a random index into the data. Compute the predicted output Ypred for Xt with the forward method. Compute the loss for Ypred relative to Yt. Use the backward method to compute the gradients. Use the sgd_step method to change the weights. Repeat.

Hint: First call `Sequential.loss.backward()` to get dLdZ (in the case of the NLL loss function) before calling `Sequential.backward` with dLdZ as your input to propagate the loss backward through the layers.

In [198]:
 class Sequential:
    def __init__(self, modules, loss):            # List of modules, loss module
        self.modules = modules
        self.loss = loss

    def sgd(self, X, Y, iters=100, lrate=0.005):  # Train
        D, N = X.shape
        sum_loss = 0
        for it in range(iters):
            rnd = np.random.randint(N)
            Xt, Yt =  X[:,rnd:rnd+1], Y[:,rnd:rnd+1]
            Ypred = self.forward(Xt)
            sum_loss += self.loss.forward(Ypred, Yt)
            dLdZ = self.loss.backward()
            self.backward(dLdZ)
            self.sgd_step(lrate)

    def forward(self, Xt):                        # Compute Ypred
        for m in self.modules: Xt = m.forward(Xt)
        return Xt

    def backward(self, delta):                    # Update dLdW and dLdW0
        # Note reversed list of modules
        for m in self.modules[::-1]: delta = m.backward(delta)

    def sgd_step(self, lrate):                    # Gradient descent step
        for m in self.modules: m.sgd_step(lrate)

    def print_accuracy(self, it, X, Y, cur_loss, every=250):
        # Utility method to print accuracy on full dataset, should
        # improve over time when doing SGD. Also prints current loss,
        # which should decrease over time. Call this on each iteration
        # of SGD!
        if it % every == 1:
            cf = self.modules[-1].class_fun
            acc = np.mean(cf(self.forward(X)) == cf(Y))
            print('Iteration =', it, '\tAcc =', acc, '\tLoss =', cur_loss, flush=True)

## Neural Network / SGD Test Cases: ##
Use Test 3 to help you debug.

In [199]:
# TEST 3: try calling these methods that train with a simple dataset
def nn_tanh_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), Tanh(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=1, lrate=0.005)
    return [np.vstack([nn.modules[0].W, nn.modules[0].W0.T]).tolist(),
            np.vstack([nn.modules[2].W, nn.modules[2].W0.T]).tolist()]


def nn_relu_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), ReLU(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=2, lrate=0.005)
    return [np.vstack([nn.modules[0].W, nn.modules[0].W0.T]).tolist(),
            np.vstack([nn.modules[2].W, nn.modules[2].W0.T]).tolist()]


def nn_pred_test():
    np.random.seed(0)
    nn = Sequential([Linear(2, 3), ReLU(), Linear(3, 2), SoftMax()], NLL())
    X, Y = super_simple_separable()
    nn.sgd(X, Y, iters=1, lrate=0.005)
    Ypred = nn.forward(X)
    return nn.modules[-1].class_fun(Ypred).tolist(), [nn.loss.forward(Ypred, Y)]


In [200]:
nn_tanh_test()

# Expected output:
# '''
# [[[1.2473733761848262, 0.2829538808226157, 0.6924193292712828],
#   [1.5845507770278007, 1.320562932207846, -0.6901721567010647],
#   [-8.47337764291184e-12, 2.6227368810847106e-09, 0.00017353185263155828]],
#  [[0.544808855557535, -0.08366117689965663],
#   [-0.06331837550937104, 0.24078409926389266],
#   [0.08677202043839037, 0.8360167748667923],
#   [-0.0037249480614718, 0.0037249480614718]]]
# '''

[[[1.2473733761848262, 0.2829538808226157, 0.6924193292712828],
  [1.5845507770278007, 1.320562932207846, -0.6901721567010647],
  [-8.47337764291184e-12, 2.6227368810847106e-09, 0.00017353185263155828]],
 [[0.544808855557535, -0.08366117689965663],
  [-0.06331837550937104, 0.24078409926389266],
  [0.08677202043839037, 0.8360167748667923],
  [-0.0037249480614718, 0.0037249480614718]]]

In [201]:
nn_relu_test()

# Expected output:
# '''
# [[[1.2421914999646917, 0.2851239946607419, 0.6905003767490479],
#   [1.5695659964519526, 1.3273884281993562, -0.6920877418422037],
#   [-0.0027754917572235106, 0.001212351486908601, -0.0005239629389906042]],
#  [[0.501769700845158, -0.040622022187279644],
#   [-0.09260786974986723, 0.27007359350438886],
#   [0.08364438851530624, 0.8391444067898763],
#   [-0.004252310922204504, 0.004252310922204505]]]
# '''

[[[1.2422381908871343, 0.2850917905905376, 0.6979154428450828],
  [1.5695971237335808, 1.32736695881922, -0.6734285638518853],
  [-0.0027599281164093696, 0.0012016167968404868, 0.003194616722684224]],
 [[0.5019699887733605, -0.040822310115482194],
  [-0.09250521279479793, 0.2699709365493196],
  [0.08371056534069252, 0.8390782299644901],
  [-0.0042231049094530945, 0.004223104909453096]]]

In [202]:
nn_pred_test()

# Expected output:
# '''
# ([0, 0, 0, 0], [8.56575061835767])
# '''

([0, 0, 0, 0], [8.435824137827966])